# <center> Creado Por Jorge Cardona</center>

In [2]:
# configura el tamano de cada celda al 100% de la pantalla
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>")) 

In [3]:
# importa las librerias que se van a necesitar en el procesamiento de datos

# importa pandas, que es una libreria para análisis de datos que proporciona unas estructuras de datos flexibles
import pandas as pd

# libreria que permite usar las operaciones matematicas sobre variables
import math

# libreria que sirve para articionar aleatoriamente los datasets
import random

# libreria que permite obtener los caracteres del codigo ASCII, con el metodo string.printable
import string

# libreria para realizar debug
import pixiedust

#libreria para expresiones regulares
import re

# libreria para revision ortografica
from spellchecker import SpellChecker
   
# importa os que permite leer variables del sistema operativo y manipular archivos del sistema
import os

# importa la libreria que permite hacer solicitudes
import requests

Pixiedust database opened successfully


In [4]:
# funcion que permite obtener el contenido delarchivo leido
def obtener_datos_de_archivo (archivo_lectura):
    
    # muestra todo el texto de la columna
    pd.set_option('display.max_colwidth', -1)

    # asigna el directorio de donde se va a leer el archivo
    archivo = archivo_lectura

    # lee el archivo y lo almacena en un dataFrame
    df = pd.read_excel(archivo)

    # reemplaza los saltos de linea con espacios en blanco
    df = df.replace('\n',' ', regex=True)

    # reemplaza los NaN con cadena vacia
    df = df.fillna('')
    
    # retorna el contenido cargado en la variable
    return df

In [5]:
# funcion que permite obtener los limites paa lectura del archivo leido
def obtener_datos_para_control_de_contenido (df_original):
    
    # lee la linea donde estan las etiquetas de la tabla
    etiquetados = df_original.iloc[:0,]

    # obtiene los nombres de las columnas para procesar
    labels = df_original.columns.values

    # listado de columnas a eliminar
    archivoDepurado= df_original.drop(columns=['ARCHIVO','COMPLEMENTACION']) # no se tienen en cuenta ARCHIVO	OCR FULL	COMPLEMENTACION

    # obtiene la cantidad de filas a recorrer
    cantidad_filas    = archivoDepurado.shape[0] 

    # obtiene la cantidad de columnas a recorrer
    cantidad_columnas = (archivoDepurado.shape[1]) 

    # obtiene la cantidad de columnas a evaluar
    cantidad_propietarios = int(cantidad_columnas/5) # se divide en 5 ya que son 5 campos por propietario, debe dar numero exacto

    # obtiene los nombres de los labels para separar las consultas
    labels = pd.Series(archivoDepurado.columns.values).tolist()
    
    return cantidad_filas,cantidad_propietarios

In [6]:
def obtener_listado_nombres_apellidos_corporaciones(fila, columnas_totales, NOMBRE_ARCHIVO_LEIDO, TEXTO_COMPLEMENTACION):
    
    # define que la etiqueta contenga como minimo los carateres definidos
    TAMANO_MINIMO_TEXTO_ETIQUETADO                 = 5  
    
    # define que los nombres y apellidos etiquetados contengan por lo menos el numero de caracteres designado
    TAMANO_MINIMO_NOMBRES_APELLIDOS_CORPORACIONES  = 3  
    
    # almacena los listados de corporaciones, nombres, apellidos leidos en la fila del archivo
    SIMBOLO_PREPROCESAMIENTO   = '▓'
    LISTADO_CORPORACIONES      = []
    LISTADO_NOMBRES            = []
    LISTADO_APELLIDOS          = [] 
    COMPLEMENTACION            = TEXTO_COMPLEMENTACION
    # preprocesamiento, cambio de caracter coma ',' por el SIMBOLO_PREPROCESAMIENTO para evitar fallos al usar el contenido de informacion en vectores
    COMPLEMENTACION            = COMPLEMENTACION.replace(',',SIMBOLO_PREPROCESAMIENTO) 
    COMPLEMENTACION            = COMPLEMENTACION.replace("\'",'').replace('\\','').replace('"', '') 
    COMPLEMENTACION_TOKENIZADA = COMPLEMENTACION
    
    
    # recorre cada columna de la fila para leer el contenido y definir si es nombre, apellido o corporacion
    for columna in range(1, columnas_totales):
        
        # complemento de la columna a leer del archivo
        ETIQUETA                         = str(columna) + '_ETIQUETA_COMPLETA'     
        
        # obtiene la columna del dataframe
        COLUMNA_LEIDA_DATAFRAME          = df_original[ETIQUETA]
        # preprocesamiento, cambio de caracter coma ',' por el SIMBOLO_PREPROCESAMIENTO para evitar fallos al usar el contenido de informacion en vectores
        COLUMNA_LEIDA_DATAFRAME          = COLUMNA_LEIDA_DATAFRAME.replace(',',SIMBOLO_PREPROCESAMIENTO) 

        # obtiene la informacion de texto etiquetado
        TEXTO_ETIQUETADO                 = COLUMNA_LEIDA_DATAFRAME.iloc[fila:fila + 1].to_string(index=False).upper()
               
        # Verifica que la columna leida X_ETIQUETA_COMPLETA tenga contenido y que al menos tenga 5 caracteres
        if((TEXTO_ETIQUETADO != '') and (len(TEXTO_ETIQUETADO) > TAMANO_MINIMO_TEXTO_ETIQUETADO)):
            
            # obtiene la informacion de cada etiqueta del archivo y la almacena en la variable correspondiente, para realizar las validaciones siguientes
            NOMBRE_1                         = df_original[str(columna) + '_ETIQUETA_NOMBRE_1']   .iloc[fila:fila + 1].to_string(index=False).upper().strip()
            NOMBRE_2                         = df_original[str(columna) + '_ETIQUETA_NOMBRE_2']   .iloc[fila:fila + 1].to_string(index=False).upper().strip()
            APELLIDO_1                       = df_original[str(columna) + '_ETIQUETA_APELLIDO_1'] .iloc[fila:fila + 1].to_string(index=False).upper().strip()
            APELLIDO_2                       = df_original[str(columna) + '_ETIQUETA_APELLIDO_2'] .iloc[fila:fila + 1].to_string(index=False).upper().strip()
            CORPORACION                      = df_original[str(columna) + '_ETIQUETA_COMPLETA']   .iloc[fila:fila + 1].to_string(index=False).upper().strip()  
            
            # preprocesamiento, cambio de caracter coma ',' por el SIMBOLO_PREPROCESAMIENTO para evitar fallos al usar el contenido de informacion en vectores
            NOMBRE_1                         = NOMBRE_1.replace(',',SIMBOLO_PREPROCESAMIENTO).replace("\'",'').replace('\\','').replace('"', '')   
            NOMBRE_2                         = NOMBRE_2.replace(',',SIMBOLO_PREPROCESAMIENTO).replace("\'",'').replace('\\','').replace('"', '')  
            APELLIDO_1                       = APELLIDO_1.replace(',',SIMBOLO_PREPROCESAMIENTO).replace("\'",'').replace('\\','').replace('"', '') 
            APELLIDO_2                       = APELLIDO_2.replace(',',SIMBOLO_PREPROCESAMIENTO).replace("\'",'').replace('\\','').replace('"', '') 
            CORPORACION                      = CORPORACION.replace(',',SIMBOLO_PREPROCESAMIENTO).replace("\'",'').replace('\\','').replace('"', '')                     
            
            # verifica si no existen NOMBRES NI APELLIDOS, asociados a la X_ETIQUETA_COMPLETA leida
            if( NOMBRE_1 == '' and NOMBRE_2 == '' and APELLIDO_1 == '' and APELLIDO_2 == ''):                  

                # valida que la etiqueta cumpla con el minimo de carateres declaradas para nombres y apellidos
                if(len(NOMBRE_1) >= TAMANO_MINIMO_NOMBRES_APELLIDOS_CORPORACIONES):
                    
                    # adiciona las corporaciones encontradas al listado de corporaciones
                    LISTADO_CORPORACIONES.append(CORPORACION)
                
            # verifica si solo existe NOMBRE_1 asociado a la X_ETIQUETA_COMPLETA leida
            elif(NOMBRE_1 != '' and NOMBRE_2 == '' and APELLIDO_1 == '' and APELLIDO_2 == ''):
                
                # valida que la etiqueta cumpla con el minimo de carateres declaradas para nombres y apellidos
                if(len(NOMBRE_1) >= TAMANO_MINIMO_NOMBRES_APELLIDOS_CORPORACIONES):

                    # adiciona las corporaciones encontradas al listado de corporaciones
                    LISTADO_CORPORACIONES.append(CORPORACION)

            # verifica si solo existen NOMBRE_1 y NOMBRE_2 asocioados a la X_ETIQUETA_COMPLETA leida
            elif( NOMBRE_1 != '' and NOMBRE_2 != '' and APELLIDO_1 == '' and APELLIDO_2 == ''):                
                               
                # valida que la etiqueta cumpla con el minimo de carateres declaradas para nombres y apellidos
                if(len(NOMBRE_1) >= TAMANO_MINIMO_NOMBRES_APELLIDOS_CORPORACIONES):
                    # adiciona los nombres encontradas al listado de nombres
                    LISTADO_NOMBRES.append(NOMBRE_1)
                    
                # valida que la etiqueta cumpla con el minimo de carateres declaradas para nombres y apellidos  
                if(len(NOMBRE_2) >= TAMANO_MINIMO_NOMBRES_APELLIDOS_CORPORACIONES):
                    # adiciona los nombres encontradas al listado de nombres
                    LISTADO_NOMBRES.append(NOMBRE_2)
                
            # verifica si solo existe NOMBRE_1 y APELLIDO_1 asociados a la X_ETIQUETA_COMPLETA leida
            elif(NOMBRE_1 != '' and NOMBRE_2 == '' and APELLIDO_1 != '' and APELLIDO_2 == ''):

                # valida que la etiqueta cumpla con el minimo de carateres declaradas para nombres y apellidos
                if(len(NOMBRE_1) >= TAMANO_MINIMO_NOMBRES_APELLIDOS_CORPORACIONES):
                    # adiciona los nombres encontradas al listado de nombres
                    LISTADO_NOMBRES.append(NOMBRE_1)
                    
                # valida que la etiqueta cumpla con el minimo de carateres declaradas para nombres y apellidos   
                if(len(APELLIDO_1) >= TAMANO_MINIMO_NOMBRES_APELLIDOS_CORPORACIONES):
                    # adiciona los apellidos encontradas al listado de apellidos
                    LISTADO_APELLIDOS.append(APELLIDO_1)
        
            # verifica si solo existen NOMBRE_1, NOMBRE_2 y APELLIDO_1 asociados a la X_ETIQUETA_COMPLETA leida
            elif( NOMBRE_1 != '' and NOMBRE_2 != '' and APELLIDO_1 != '' and APELLIDO_2 == ''):                 
               
                # valida que la etiqueta cumpla con el minimo de carateres declaradas para nombres y apellidos
                if(len(NOMBRE_1) >= TAMANO_MINIMO_NOMBRES_APELLIDOS_CORPORACIONES):
                    # adiciona los nombres encontradas al listado de nombres
                    LISTADO_NOMBRES.append(NOMBRE_1)
                    
                # valida que la etiqueta cumpla con el minimo de carateres declaradas para nombres y apellidos
                if(len(NOMBRE_2) >= TAMANO_MINIMO_NOMBRES_APELLIDOS_CORPORACIONES):
                    # adiciona los nombres encontradas al listado de nombres
                    LISTADO_NOMBRES.append(NOMBRE_2)
                    
                # valida que la etiqueta cumpla con el minimo de carateres declaradas para nombres y apellidos   
                if(len(APELLIDO_1) >= TAMANO_MINIMO_NOMBRES_APELLIDOS_CORPORACIONES):
                    # valida que la etiqueta cumpla con el minimo de carateres declaradas para nombres y apellidos
                    LISTADO_APELLIDOS.append(APELLIDO_1)

            # verifica si solo existe NOMBRE_2 y el APELLIDO_1 asociados a la X_ETIQUETA_COMPLETA leida
            elif( NOMBRE_1 == '' and NOMBRE_2 != '' and APELLIDO_1 != '' and APELLIDO_2 == ''):                 
               
                # valida que la etiqueta cumpla con el minimo de carateres declaradas para nombres y apellidos
                if(len(NOMBRE_2) >= TAMANO_MINIMO_NOMBRES_APELLIDOS_CORPORACIONES):
                    # adiciona los nombres encontradas al listado de nombres
                    LISTADO_NOMBRES.append(NOMBRE_2)

                # valida que la etiqueta cumpla con el minimo de carateres declaradas para nombres y apellidos   
                if(len(APELLIDO_1) >= TAMANO_MINIMO_NOMBRES_APELLIDOS_CORPORACIONES):
                    # valida que la etiqueta cumpla con el minimo de carateres declaradas para nombres y apellidos
                    LISTADO_APELLIDOS.append(APELLIDO_1)

            # verifica si solo existe NOMBRE_2 y los aAPELLIDO_1 y APELLIDO_2 a la X_ETIQUETA_COMPLETA leida
            elif( NOMBRE_1 == '' and NOMBRE_2 != '' and APELLIDO_1 != '' and APELLIDO_2 != ''):                 
               
                # valida que la etiqueta cumpla con el minimo de carateres declaradas para nombres y apellidos
                if(len(NOMBRE_2) >= TAMANO_MINIMO_NOMBRES_APELLIDOS_CORPORACIONES):
                    # adiciona los nombres encontradas al listado de nombres
                    LISTADO_NOMBRES.append(NOMBRE_2)
                    
                # valida que la etiqueta cumpla con el minimo de carateres declaradas para nombres y apellidos   
                if(len(APELLIDO_1) >= TAMANO_MINIMO_NOMBRES_APELLIDOS_CORPORACIONES):
                    # valida que la etiqueta cumpla con el minimo de carateres declaradas para nombres y apellidos
                    LISTADO_APELLIDOS.append(APELLIDO_1)
                    
                # valida que la etiqueta cumpla con el minimo de carateres declaradas para nombres y apellidos   
                if(len(APELLIDO_2) >= TAMANO_MINIMO_NOMBRES_APELLIDOS_CORPORACIONES):
                    # valida que la etiqueta cumpla con el minimo de carateres declaradas para nombres y apellidos
                    LISTADO_APELLIDOS.append(APELLIDO_2)
                    
            # verifica si solo existe NOMBRE_1 y los dos apellidos asocioados a la X_ETIQUETA_COMPLETA leida
            elif(NOMBRE_1 != '' and NOMBRE_2 == '' and APELLIDO_1 != '' and APELLIDO_2 != ''):
                
                # valida que la etiqueta cumpla con el minimo de carateres declaradas para nombres y apellidos
                if(len(NOMBRE_1) >= TAMANO_MINIMO_NOMBRES_APELLIDOS_CORPORACIONES):
                    # adiciona los nombres encontradas al listado de nombres
                    LISTADO_NOMBRES.append(NOMBRE_1)

                # valida que la etiqueta cumpla con el minimo de carateres declaradas para nombres y apellidos   
                if(len(APELLIDO_1) >= TAMANO_MINIMO_NOMBRES_APELLIDOS_CORPORACIONES):
                    # adiciona los apellidos encontradas al listado de apellidos
                    LISTADO_APELLIDOS.append(APELLIDO_1)
                    
                 # valida que la etiqueta cumpla con el minimo de carateres declaradas para nombres y apellidos   
                if(len(APELLIDO_2) >= TAMANO_MINIMO_NOMBRES_APELLIDOS_CORPORACIONES):
                    # adiciona los apellidos encontradas al listado de apellidos
                    LISTADO_APELLIDOS.append(APELLIDO_2)
                                
            # tiene ambos nombres y apellidos en la etiqueta leida
            else:

                # valida que la etiqueta cumpla con el minimo de carateres declaradas para nombres y apellidos
                if(len(NOMBRE_1) >= TAMANO_MINIMO_NOMBRES_APELLIDOS_CORPORACIONES):
                    # adiciona los nombres encontradas al listado de nombres
                    LISTADO_NOMBRES.append(NOMBRE_1)
                    
                # valida que la etiqueta cumpla con el minimo de carateres declaradas para nombres y apellidos
                if(len(NOMBRE_2) >= TAMANO_MINIMO_NOMBRES_APELLIDOS_CORPORACIONES):
                    # adiciona los nombres encontradas al listado de nombres
                    LISTADO_NOMBRES.append(NOMBRE_2)
                    
                # valida que la etiqueta cumpla con el minimo de carateres declaradas para nombres y apellidos   
                if(len(APELLIDO_1) >= TAMANO_MINIMO_NOMBRES_APELLIDOS_CORPORACIONES):
                    # valida que la etiqueta cumpla con el minimo de carateres declaradas para nombres y apellidos
                    LISTADO_APELLIDOS.append(APELLIDO_1)
                    
                 # valida que la etiqueta cumpla con el minimo de carateres declaradas para nombres y apellidos   
                if(len(APELLIDO_2) >= TAMANO_MINIMO_NOMBRES_APELLIDOS_CORPORACIONES):
                    # valida que la etiqueta cumpla con el minimo de carateres declaradas para nombres y apellidos
                    LISTADO_APELLIDOS.append(APELLIDO_2)
                
        # elimina los elementos repetidos en las listas
        LISTADO_CORPORACIONES = list(set(LISTADO_CORPORACIONES))
        LISTADO_NOMBRES       = list(set(LISTADO_NOMBRES))
        LISTADO_APELLIDOS     = list(set(LISTADO_APELLIDOS))

        # ordena los elementos de las listas de mayor a menor tamano en caracteres
        LISTADO_CORPORACIONES = sorted(LISTADO_CORPORACIONES,key=len, reverse=True)
        LISTADO_NOMBRES       = sorted(LISTADO_NOMBRES,key=len, reverse=True)
        LISTADO_APELLIDOS     = sorted(LISTADO_APELLIDOS,key=len, reverse=True)

        # lista especifica que permite guardar una cadena de cada elemento de la lista 'CORPORACIONES','NOMBRES','APELLIDOS'
        # en forma tokenizada, para reemplazarlo posteriomente en el texto de la complementacion
        LISTADO_CORPORACIONES_REEMPLAZABLE        = []
        LISTADO_NOMBRES_REEMPLAZABLE              = []
        LISTADO_APELLIDOS_REEMPLAZABLE            = []
        LISTA_CORPORACIONES                       = []
        LISTA_APELLIDOS                           = []
        LISTA_NOMBRES                             = []
        LISTADO_DE_LISTAS_ITERABLES               = [LISTA_CORPORACIONES, LISTA_APELLIDOS, LISTA_NOMBRES]
        LISTA_DE_UNION_ORIGINALES_Y_TOKENIZADOS   = []

        # variables declaradas para ser la base de los reemplazos tokenizados de cada lista usada
        ITERADOR_CORPORACIONES_NOMBRES_APELLIDOS  = [LISTADO_CORPORACIONES,LISTADO_NOMBRES,LISTADO_APELLIDOS]
        LISTADO_BASE_REEMPLAZO                    = ['CORPORACIONES','NOMBRES','APELLIDOS']
        SIMBOLOS_BASE_REEMPLAZO                   = ['Ξ','Ψ','Ϡ']
        LISTADOS_REEMPLAZABLES                    = [LISTADO_CORPORACIONES_REEMPLAZABLE,LISTADO_NOMBRES_REEMPLAZABLE,LISTADO_APELLIDOS_REEMPLAZABLE]

        # recorre los listados uno a uno, corporaciones, nombres, apellidos
        for iterador_de_todos_los_listados in range(len(ITERADOR_CORPORACIONES_NOMBRES_APELLIDOS)):

            # recorre uno a uno los elementos de cada listado primero los de corporaciones, luego los de apellidos, y luego los de nombres
            # crea unas cadenas del tamano de cada corporacion, apellido y nombres, con los simbolos correspondientes a 'CORPORACIONES','NOMBRES','APELLIDOS'
            for iterador_de_listado_seleccionado in range(len(ITERADOR_CORPORACIONES_NOMBRES_APELLIDOS[iterador_de_todos_los_listados])):

                # variable con contenido vacio, para luego crear
                REEMPLAZO = ''

                # lee el token que se va a usar como reemplazo en cada una de las listas
                TOKEN_REEMPLAZO    = SIMBOLOS_BASE_REEMPLAZO[iterador_de_todos_los_listados]

                # carga cual lista es la que se va a generar con los token de reemplazo
                LISTA_REEMPLAZABLE = LISTADOS_REEMPLAZABLES[iterador_de_todos_los_listados]

                # crea una cadena de igual tamaño para cada elemento de la lista seleccionada, con el simbolo seleccionado
                for listado_final in range(len(ITERADOR_CORPORACIONES_NOMBRES_APELLIDOS[iterador_de_todos_los_listados][iterador_de_listado_seleccionado])):
                    REEMPLAZO += TOKEN_REEMPLAZO

                # une el contenido generado de los elementos de cada lista
                LISTA_REEMPLAZABLE.append(REEMPLAZO)             
        
        # variables declaradascomo base para reemplazar los valores orginales con los tokens en la complementacion
        LISTADO_DATOS_ORIGINALES_DE_COMPLEMENTACION     = [LISTADO_CORPORACIONES,LISTADO_APELLIDOS,LISTADO_NOMBRES]
        LISTADO_DATOS_PARA_REEMPLAZO_EN_COMPLEMENTACION = [LISTADO_CORPORACIONES_REEMPLAZABLE,LISTADO_APELLIDOS_REEMPLAZABLE,LISTADO_NOMBRES_REEMPLAZABLE]
        
        # itera cada lista original y tokenizada y une los valores de sus indices en un vector
        for lista_datos_originales_leida in range(len(LISTADO_DATOS_ORIGINALES_DE_COMPLEMENTACION)):
            
            listapegada = LISTADO_DE_LISTAS_ITERABLES[lista_datos_originales_leida]
            # itera los valores contenidos dentro de cada leida corporaciones,apellidos, nombres
            for sublista_leida in range (len(LISTADO_DATOS_ORIGINALES_DE_COMPLEMENTACION[lista_datos_originales_leida])):
                
                # lee los valores de las listas con datos originales y con listas de datos de remplazo
                valor_lista_original   = LISTADO_DATOS_ORIGINALES_DE_COMPLEMENTACION[lista_datos_originales_leida][sublista_leida]
                valor_lista_remplazo   = LISTADO_DATOS_PARA_REEMPLAZO_EN_COMPLEMENTACION[lista_datos_originales_leida][sublista_leida]
                
                # reemplaza los datos en la complementacion, basado en los datos leidos
                # de la lista original y la de remplazo
                
                listapegada.append(valor_lista_original + ',' + valor_lista_remplazo)
                
        # une todas los datos generados por cada lista
        LISTA_DE_UNION_ORIGINALES_Y_TOKENIZADOS = LISTA_CORPORACIONES + LISTA_APELLIDOS + LISTA_NOMBRES
                
    # ordena de mayor a menor cada la lista que unio todas las listas
    LISTA_DE_UNION_ORIGINALES_Y_TOKENIZADOS = sorted(LISTA_DE_UNION_ORIGINALES_Y_TOKENIZADOS,key=len, reverse=True)

    # itera la lista de palabras originales etiquetadas y sus versiones de reemplazo, y sustituye las frases originales por las de reemplazo
    for posicion_lista_final_tokenizada in range (len(LISTA_DE_UNION_ORIGINALES_Y_TOKENIZADOS)):
        
        # lee los valores de las listas con datos originales y con listas de datos de remplazo
        lista_reemplazo_final  = LISTA_DE_UNION_ORIGINALES_Y_TOKENIZADOS[posicion_lista_final_tokenizada].split(',')
        
        # obtiene el valor original y el de reemplazo para la complementacion
        valor_lista_original   = lista_reemplazo_final[0]
        valor_lista_remplazo   = lista_reemplazo_final[1]
        
        # reemplaza los datos en la complementacion, basado en los datos leidos
        # de la lista original y la de remplazo
        COMPLEMENTACION_TOKENIZADA = COMPLEMENTACION_TOKENIZADA.replace(valor_lista_original , valor_lista_remplazo)     

    # devuelve el listado de nombres, apellidos y corporaciones leidas en la fila del archivo, 
    # ordenados por tamano, sin repetir y tambien en sus formas de reemplazo tokenizadas
    return LISTADO_CORPORACIONES, LISTADO_CORPORACIONES_REEMPLAZABLE, LISTADO_APELLIDOS, LISTADO_APELLIDOS_REEMPLAZABLE, LISTADO_NOMBRES, LISTADO_NOMBRES_REEMPLAZABLE, COMPLEMENTACION, COMPLEMENTACION_TOKENIZADA

In [7]:
# funcion que recorre todas las filas del archivo y llama a la funcion que recorre las columnas de cada fila
def recorrer_filas_para_obtener_listados (fila_inicial,fila_final,columnas_totales):

    # almacena los listados obtenidos de nombres, apellidos y organizaciones para luego usarlos como diccionarios
    LISTADOS_CORPORACIONES_NOMBRES_APELLIDOS_COMPLEMENTACIONES_Y_VALORES_TOKENIZADOS = []
    
    # recorre las filas del dataframe
    for fila in range(fila_inicial,fila_final):       

        SIMBOLO_PREPROCESAMIENTO   = '▓'
        TEXTO_COMPLEMENTACION      = df_original['COMPLEMENTACION'].iloc[fila].upper().strip()
        # preprocesamiento, cambio de caracter coma ',' por el SIMBOLO_PREPROCESAMIENTO para evitar fallos al usar el contenido de informacion en vectores
        TEXTO_COMPLEMENTACION      = TEXTO_COMPLEMENTACION.replace(',',SIMBOLO_PREPROCESAMIENTO)  
        TEXTO_COMPLEMENTACION      = TEXTO_COMPLEMENTACION.replace("\'",'').replace('\\','').replace('"', '')    
        NOMBRE_ARCHIVO_LEIDO       = df_original['ARCHIVO'].iloc[fila].upper().strip()

        # llama a la funcion resultados_recorrer_columnas para obtener la informacion de las columnas de cada fila
        resultados_obtencion_listados = obtener_listado_nombres_apellidos_corporaciones(fila, columnas_totales, NOMBRE_ARCHIVO_LEIDO, TEXTO_COMPLEMENTACION)
        
        # obtiene los resultados de los listados y los almacena en una lista
        LISTADOS_CORPORACIONES_NOMBRES_APELLIDOS_COMPLEMENTACIONES_Y_VALORES_TOKENIZADOS.append(resultados_obtencion_listados)
        
    # retorna la lista de nombres, apellidos y organizaciones de cada OCR leido
    return LISTADOS_CORPORACIONES_NOMBRES_APELLIDOS_COMPLEMENTACIONES_Y_VALORES_TOKENIZADOS

In [8]:
# funcion que convierte la complementacion de los OCR en listas que son separadas cada que hay un espacio en blanco
def preparar_complementacion_de_ocr_tokenizado_por_palabras_clave (LISTADO_OCR_TOKENIZADOS):
    
    # variable que almacena toda la complementacion
    TEXTO_DEPURADO = []
    # guarda los contenidos de cada OCR particionado
    for ocr_numero in range(len(LISTADO_OCR_TOKENIZADOS)):
        
        # fracciona el texto cada que encuentre un espacio
        TEXTO_DEPURADO.append(LISTADO_OCR_TOKENIZADOS[ocr_numero].split())
        
    # retorna el texto de la complementacion partido en forma de lista
    return TEXTO_DEPURADO

In [9]:
# funcion que convierte la complementacion de los OCR en listas que son separadas cada que hay un espacio en blanco
def preparar_complementacion_de_ocr_tokenizado_por_palabras_clave_coordenadas (LISTADO_OCR_TOKENIZADOS):
    
    # variable que almacena toda la complementacion
    TEXTO_DEPURADO = []
    # guarda los contenidos de cada OCR particionado
    for ocr_numero in range(len(LISTADO_OCR_TOKENIZADOS)):
        TEXTO_DEPURADO.append(LISTADO_OCR_TOKENIZADOS[ocr_numero].split())
        
    # retorna el texto de la complementacion partido en forma de lista
    return TEXTO_DEPURADO

In [10]:
# obtiene todas las complementaciones tokenizadas en forma de lista y tambien en forma de string
def obtener_listas_con_palabras_clave (LISTADO_OCR_TOKENIZADOS): 
        
    # va a almacenar todos los OCR particionados
    OCR_ENTRENAMIENTO_LISTA_DE_LISTAS = []

    # declara los simbolos por los cuales se va a reeemplazar cada tipo de contenido encontrado
    VERIFICAR      = ['ΨΨ','ϠϠ','ΞΞ']
    # declara las palabras clave con que se reemplazan los simbolos
    ACTUALIZAR     = ['NOMBRE','APELLIDO','ORGANIZACION','☹']

    # recorre la lista de listas de cada elemento de la lista creada para cada OCR
    for ocr_leido in range(len(preparar_complementacion_de_ocr_tokenizado_por_palabras_clave(LISTADO_OCR_TOKENIZADOS))):

        # lee la complementacion tokenizada que va a ser reemplazada por las palabras clave organizacion, nombre, apellido, carita triste '☹', en una lista
        OCR_A_TRATAR = preparar_complementacion_de_ocr_tokenizado_por_palabras_clave(LISTADO_OCR_TOKENIZADOS)[ocr_leido]    

        # define la posicion donde inicia la lectura de cada palabra comparada 
        POSICION_INICIAL   = 0
        
        # reemplaza los simbolos encontrados por las palabras clave definidas
        for index, item in enumerate(OCR_A_TRATAR):

            # verifica si es un nombre muestra la ubicacion donde termina la palabra ,adiciona el simbolo de reemplazo 'NOMBRE', concatena '=' con el tamaño del texto leido y le adiciona las posiciones donde esta la palabra leida
            if (item.find(VERIFICAR[0])) >= 0 :
                OCR_A_TRATAR[index] = str(POSICION_INICIAL + len(item) - 1) + '⇆' + ACTUALIZAR[0] + '=' + str(len(item)) + '[' + str(POSICION_INICIAL) + ':'  + str(POSICION_INICIAL + len(item) - 1) + ']'
                #POSICION_INICIAL = len(item)
                
            # verifica si es un apellido muestra la ubicacion donde termina la palabra ,adiciona el simbolo de reemplazo 'APELLIDO', concatena '=' con el tamaño del texto leido y le adiciona las posiciones donde esta la palabra leida
            elif (item.find(VERIFICAR[1])) >= 0 :
                OCR_A_TRATAR[index] = str(POSICION_INICIAL + len(item) - 1) + '⇆' + ACTUALIZAR[1] + '=' + str(len(item)) + '[' + str(POSICION_INICIAL) + ':'  + str(POSICION_INICIAL + len(item) - 1) + ']'
                #POSICION_INICIAL = len(item)
                
            # verifica si es una organizacion muestra la ubicacion donde termina la palabra ,adiciona el simbolo de reemplazo 'ORGANIZACION', concatena '=' con el tamaño del texto leido y le adiciona las posiciones donde esta la palabra leida
            elif (item.find(VERIFICAR[2])) >= 0 :
                OCR_A_TRATAR[index] = str(POSICION_INICIAL + len(item) - 1) + '⇆' + ACTUALIZAR[2] + '=' + str(len(item)) + '[' + str(POSICION_INICIAL) + ':'  + str(POSICION_INICIAL + len(item) - 1) + ']'
                #POSICION_INICIAL = len(item)
                
            # como no es ninguna opcion, muestra la ubicacion donde termina la palabra ,adiciona el simbolo de reemplazo '☹', conconcatena '=' con el tamaño del texto leido y le adiciona las posiciones donde esta la palabra leida
            else:        
                OCR_A_TRATAR[index] = str(POSICION_INICIAL + len(item) - 1) + '⇆' + ACTUALIZAR[3] + '=' + str(len(item)) + '[' + str(POSICION_INICIAL) + ':'  + str(POSICION_INICIAL + len(item) - 1) + ']'
                #POSICION_INICIAL = len(item)
             
            # actualiza la posicion del contenido leido y se le suma 1 que es el espacio en blanco entre la palabra actual y la proxima palabra 
            POSICION_INICIAL += len(item) + 1
                
        # agrupa los contenidos actualizados en una lista
        OCR_ENTRENAMIENTO_LISTA_DE_LISTAS.append(OCR_A_TRATAR)

    # retorna la lista actualizada con los valores organizacion, nombre, apellido, carita triste '☹', en una lista
    return OCR_ENTRENAMIENTO_LISTA_DE_LISTAS

In [11]:
# funcion que obtiene todos las complementaciones de los OCR originales y las complementaciones tokenizadas
def obtener_ocr_originales_y_ocr_tokenizados (LISTADOS_OCR_DE_CORPORACIONES_NOMBRES_APELLIDOS_COMPLEMENTACIONES_Y_VALORES_TOKENIZADOS, TOTAL_OCR_LEIDOS):
    
    # variable que almacena las complementaciones en forma de lista
    LISTADO_OCR_ORIGINALES  = []
    LISTADO_OCR_TOKENIZADOS = []
    
    # recorre lalista que tiene guardado los datos de la complementacion original y tokenizada
    for ocr_leido in range(TOTAL_OCR_LEIDOS):
        
        # esta es la posicion donde se encuentran ubicados los datos en el return de la funcion recorrer_filas_para_obtener_listados
        # y donde el resultado se encuentra contenido en la variable LISTADOS_OCR_DE_CORPORACIONES_NOMBRES_APELLIDOS_COMPLEMENTACIONES_Y_VALORES_TOKENIZADOS
        posicion_ocr_original   = 6
        posicion_ocr_tokenizado = 7
        
        # adiciona una a una las complementaciones que son leidas
        LISTADO_OCR_ORIGINALES.append(LISTADOS_OCR_DE_CORPORACIONES_NOMBRES_APELLIDOS_COMPLEMENTACIONES_Y_VALORES_TOKENIZADOS[ocr_leido][posicion_ocr_original])
        LISTADO_OCR_TOKENIZADOS.append(LISTADOS_OCR_DE_CORPORACIONES_NOMBRES_APELLIDOS_COMPLEMENTACIONES_Y_VALORES_TOKENIZADOS[ocr_leido][posicion_ocr_tokenizado])
    
    # retorna el listado de las complementaciones originales y tokenizadas
    return LISTADO_OCR_ORIGINALES, LISTADO_OCR_TOKENIZADOS
    

In [12]:
# permite dividir las complementaciones en fracciones de texto utilizables para entrenamiento
def fraccionamiento_de_texto_en_complementacion_usando_tokens (LISTADO_OCR_ORIGINALES, LISTADO_OCR_TOKENIZADOS ):
    
    #variable declara para guardar todas las fracciones de texto TOKENIZADAS de los ocr leidos
    complementaciones_fraccionadas_originales  = []
    
    # guarda todo los tokenizados con posiciones del original
    complementaciones_fraccionadas_tokenizadas = []

    # comienza a recorrer todas las complementaciones guardadas de los OCR
    for complementacion_leida in range(len(LISTADO_OCR_ORIGINALES)):

        # guarda la complementacion que se va a fraccionar
        contenido_complementacion_original   = LISTADO_OCR_ORIGINALES[complementacion_leida] 
        contenido_complementacion_tokenizada = LISTADO_OCR_TOKENIZADOS[complementacion_leida]

        # guarda la complementacion que se va a fraccionar
        texto_actualizado         = contenido_complementacion_original

        # declara el maximo valor de caracteres con que se DEBEN CREAR las frases , que son validas para las herramientas a trabajar
        # este es un parametro estandar
        limite_maximo_permitido    = 500

        # declara el minimo valor de caracteres en que se BUSCAN LOS TOKEN Y CON ESO SE CREAN frases, que son validas para las herramientas a trabajar
        # este es un parametro estandar
        limite_minimo_permitido    = 490

        ################################################################################################################################################################
        ################################################################################################################################################################
        ############## LA SIGUIENTE VARIABLE 'limite_frase_valida' ES PARA INGRESAR EL VALOR DEL TAMANO DE LAS FRASES QUE SE QUIEREN CREAR #############################
        ################################################################################################################################################################
        ################################################################################################################################################################
        
        # se define la cantidad de caracteres maximos en que se busca particionar la complementacion    
        limite_frase_valida       = 150

        # valida que la fraccion de texto a crear no sea muy pequena, menor de 150 caracteres, de ser asi se altera el limite de particiones necesarias en el texto
        if(limite_frase_valida < limite_minimo_permitido):

            # reasigna el valor minimo de particion de frases, basado en los parametros, confirurados como estandar
            limite_frase_valida = limite_minimo_permitido

            # calcula basado en la cantidad de caracteres a fraccionar, cuanto deberia ser el numero de recorrido maximo para completar la particion del texto
            limite_particion_de_texto = math.ceil(len(contenido_complementacion_original)/limite_frase_valida) * 2

        # valida que no supere la capacidad maxima permitida
        elif(limite_frase_valida > limite_maximo_permitido):
            # se define la cantidad de caracteres maximos en que se busca particionar la complementacion    
            limite_frase_valida       = limite_maximo_permitido

            # calcula basado en la cantidad de caracteres a fraccionar, cuanto deberia ser el numero de recorrido maximo para completar la particion del texto        
            limite_particion_de_texto = math.ceil(len(contenido_complementacion_original)/limite_frase_valida) * 2

        # asigna el limite de particiones, asegurando que esta hasta el rango maximo permitido para la particion de frases
        else:
            # calcula basado en la cantidad de caracteres a fraccionar, cuanto deberia ser el numero de recorrido maximo para completar la particion del texto        
            limite_particion_de_texto = math.ceil(len(contenido_complementacion_original)/limite_frase_valida) * 2

        # toma el tamano que se quiere
        texto_complementacion_actualizado_original   = contenido_complementacion_original
        texto_complementacion_actualizado_tokenizado = contenido_complementacion_tokenizada

        # guarda el contenido de la complementacion particionada EN UNA LISTA
        union_texto_orginal     = []
        union_texto_tokenizado  = []

        # almacena el contenido de la porcion de texto que se quiere verificar de tamano y ver si es necesario particionar con TOKEN 
        frase_original_a_evaluar   = '1'
        frase_tokenizada_a_evaluar = '1'

        #print('TAMANO COMPLEMENTACION = ', len(contenido_complementacion_original))
        #print('FUE NECESARIO PARTIR EN = ', limite_particion_de_texto , ' PARTES')
        #print()
        #print(contenido_complementacion_original)    
        #print()

        # itera la complementacion para separarla en textos fraccionados, con el tamano que se desea
        for particionado in range(limite_particion_de_texto):

            # guarda la complementacion actualizada en contenido y tamano
            texto_complementacion_actualizado_original   = texto_complementacion_actualizado_original
            texto_complementacion_actualizado_tokenizado = texto_complementacion_actualizado_tokenizado

            # criterio de parada para las iteraciones, si la frase tokenizada tiene un tamano de cero, es porque no hay que seguir iterando
            if(len(frase_original_a_evaluar) > 0):            

                # quiere decir que hay contenido que se puede TOKENIZAR o guardar directamente
                # if validacion, si existe contenido en texto_complementacion_actualizado_original
                if (len(texto_complementacion_actualizado_original) > 0):

                    #print('frase_original_a_evaluar ', len(frase_original_a_evaluar)) 
                    # si el texto leido de la complementacion es menor o igual al limite permitido, se guarda
                    # if validacion que el texto a evaluar este en el rango permitido
                    if(len(texto_complementacion_actualizado_original) <= limite_frase_valida):

                        #print('NO FUE NECESARIO TOKENIZADO')                     
                        #print('LA PARTICION = ', particionado + 1 , ', TIENE UN TAMANO FRASE TOKENIZADA DE :', len(texto_complementacion_actualizado_original), 'CARACTERES')
                        #print('EL UTTERANCE PARA ENTRENAMIENTO ES : ', texto_complementacion_actualizado_original)

                        # guarda el texto que cumple con el tamano en una lista
                        union_texto_orginal.append(texto_complementacion_actualizado_original)
                        union_texto_tokenizado.append(texto_complementacion_actualizado_tokenizado)
                        

                        #actualiza el tamano del texto a leer, a cero para que no se use mas
                        texto_complementacion_actualizado_original   = texto_complementacion_actualizado_original[:0]
                        texto_complementacion_actualizado_tokenizado = texto_complementacion_actualizado_tokenizado[:0]

                        # guarda el contenido del texto_complementacion_actualizado_original, en esta variable para poder verlo en el print
                        # despues de que no se necesite validar con print, se puede eliminar esta variable
                        frase_original_a_evaluar   = texto_complementacion_actualizado_original 
                        frase_tokenizada_a_evaluar = texto_complementacion_actualizado_tokenizado

                    # si el tamano del texto que existe el la complementacion es mayor
                    # al limite permitido por frase, este debe ser particionado con TOKEN
                    # else del if validacion que el texto a evaluar este en el rango permitido
                    else:
                        # define la posicion donde inicia la porcion de texto que se va a evaluar para la particion con el TOKEN
                        limite_inferior_de_texto_a_seleccionar = 0
                        # define la posicion donde finaliza la porcion de texto que se va a evaluar para la particion con el TOKEN
                        limite_superior_de_texto_a_seleccionar = limite_inferior_de_texto_a_seleccionar + limite_frase_valida
                        # toma el texto que se va a evaluar, basado en los rangos anteriores
                        frase_seleccionada_del_texto           = texto_complementacion_actualizado_original[limite_inferior_de_texto_a_seleccionar : limite_superior_de_texto_a_seleccionar]
                        # invierte el texto para buscar la aparicion del TOKEN,desde el final hasta el comienzo
                        frase_seleccionada_del_texto_invertida = frase_seleccionada_del_texto[::-1]
                        # encuentra la posicion del token en el texto invertido
                        posicion_token_encontrado              = frase_seleccionada_del_texto_invertida.find(' .')
                        # selecciona la frase particionada con la posicion del TOKEN utilizado, PUNTO ESPACIO. se pone ' .' porque el texto esta invertido, pero en el texto normal es punto espacio
                        frase_original_a_evaluar               = texto_complementacion_actualizado_original[limite_inferior_de_texto_a_seleccionar : limite_superior_de_texto_a_seleccionar - posicion_token_encontrado]
                        frase_tokenizada_a_evaluar             = texto_complementacion_actualizado_tokenizado[limite_inferior_de_texto_a_seleccionar : limite_superior_de_texto_a_seleccionar - posicion_token_encontrado]

                        # obtiene el ultimo caracter de la particion de la complementacion que se realizo
                        ultimo_caracter_frase_tokenizada       = frase_original_a_evaluar[-1:]


                        # verifica si el TOKEN utilizado para partir la frase sirve para crear una frase dentro del limite de tamano permitido, valida que el ultimo carater sea un espacio en blanco, y de ser asi guarda la frase
                        # valida el primer criterio usado como TOKEN de particion
                        if( (len(frase_original_a_evaluar) >= limite_frase_valida)) and (ultimo_caracter_frase_tokenizada != ' '):

                            # define la posicion donde inicia la porcion de texto que se va a evaluar para la particion con el TOKEN
                            limite_inferior_de_texto_a_seleccionar = 0
                            # define la posicion donde finaliza la porcion de texto que se va a evaluar para la particion con el TOKEN
                            limite_superior_de_texto_a_seleccionar = limite_inferior_de_texto_a_seleccionar + limite_frase_valida
                            # toma el texto que se va a evaluar, basado en los rangos anteriores
                            frase_seleccionada_del_texto           = texto_complementacion_actualizado_original[limite_inferior_de_texto_a_seleccionar : limite_superior_de_texto_a_seleccionar]
                            # invierte el texto para buscar la aparicion del TOKEN,desde el final hasta el comienzo
                            frase_seleccionada_del_texto_invertida = frase_seleccionada_del_texto[::-1]
                            # encuentra la posicion del token en el texto invertido
                            posicion_token_encontrado              = frase_seleccionada_del_texto_invertida.find('.')
                            # selecciona la frase particionada con la posicion del TOKEN utilizado, PUNTO
                            frase_original_a_evaluar               = texto_complementacion_actualizado_original[limite_inferior_de_texto_a_seleccionar : limite_superior_de_texto_a_seleccionar - posicion_token_encontrado]
                            frase_tokenizada_a_evaluar             = texto_complementacion_actualizado_tokenizado[limite_inferior_de_texto_a_seleccionar : limite_superior_de_texto_a_seleccionar - posicion_token_encontrado]
                            ultimo_caracter_frase_tokenizada       = frase_original_a_evaluar[-1:]

                            # verifica si el TOKEN utilizado para partir la frase sirve para crear una frase dentro del limite de tamano permitido, valida que el ultimo carater sea un espacio en blanco, y de ser asi guarda la frase
                            # valida el segundo criterio usado como TOKEN de particion
                            if( (len(frase_original_a_evaluar) >= limite_frase_valida)) and (ultimo_caracter_frase_tokenizada != ' '):

                                # define la posicion donde inicia la porcion de texto que se va a evaluar para la particion con el TOKEN
                                limite_inferior_de_texto_a_seleccionar = 0
                                # define la posicion donde finaliza la porcion de texto que se va a evaluar para la particion con el TOKEN
                                limite_superior_de_texto_a_seleccionar = limite_inferior_de_texto_a_seleccionar + limite_frase_valida
                                # toma el texto que se va a evaluar, basado en los rangos anteriores
                                frase_seleccionada_del_texto           = texto_complementacion_actualizado_original[limite_inferior_de_texto_a_seleccionar : limite_superior_de_texto_a_seleccionar]
                                # invierte el texto para buscar la aparicion del TOKEN,desde el final hasta el comienzo
                                frase_seleccionada_del_texto_invertida = frase_seleccionada_del_texto[::-1]
                                # encuentra la posicion del token en el texto invertido, ESPACIO EN BLANCO
                                posicion_token_encontrado              = frase_seleccionada_del_texto_invertida.find(' ')
                                # selecciona la frase particionada con la posicion del TOKEN utilizado
                                frase_original_a_evaluar               = texto_complementacion_actualizado_original[limite_inferior_de_texto_a_seleccionar : limite_superior_de_texto_a_seleccionar - posicion_token_encontrado]
                                frase_tokenizada_a_evaluar             = texto_complementacion_actualizado_tokenizado[limite_inferior_de_texto_a_seleccionar : limite_superior_de_texto_a_seleccionar - posicion_token_encontrado]
                                ultimo_caracter_frase_tokenizada       = frase_original_a_evaluar[-1:]                

                                # verifica si el TOKEN utilizado para partir la frase sirve para crear una frase dentro del limite de tamano permitido, valida que el ultimo carater sea un espacio en blanco, y de ser asi guarda la frase
                                # valida el tercer criterio usado como TOKEN de particion
                                if( (len(frase_original_a_evaluar) >= limite_frase_valida)) and (ultimo_caracter_frase_tokenizada != ' '):
                                    print('NO SE PUDO TOKENIZAR. ESA FRASE ESTA MUY DIFICIL, AUMENTE EL VALOR DEL LIMITE DE TAMANO MINIMO DE FRASE')                        

                                # la particion del texto con el TOKEN permitio llegar al limite de caracteres permitidos por frase
                                # else de if que valida el segundo criterio usado como TOKEN de particio
                                else:
                                    #print('TOKENIZADO CON ESPACIO EN BLANCO')
                                    # guarda el texto que cumple con el tamano en una lista
                                    union_texto_orginal.append(frase_original_a_evaluar)
                                    union_texto_tokenizado.append(frase_tokenizada_a_evaluar)
                                    # toma la cantidad de carateres
                                    posicion_de_inicio_nuevo_texto_actualizado = len(frase_original_a_evaluar)
                                    # actualiza el contenido y el tamano del texto para que sea reevaluado, si ya no mas contenido, si necesita mas particiones con token, o tiene el tamano permitido y ya se puede guardar
                                    texto_complementacion_actualizado_original   = texto_complementacion_actualizado_original[posicion_de_inicio_nuevo_texto_actualizado:]
                                    texto_complementacion_actualizado_tokenizado = texto_complementacion_actualizado_tokenizado[posicion_de_inicio_nuevo_texto_actualizado:]

                            # la particion del texto con el TOKEN permitio llegar al limite de caracteres permitidos por frase
                            # else de if que valida el segundo criterio usado como TOKEN de particio
                            else:
                                #print('TOKENIZADO CON PUNTO')
                                # guarda el texto que cumple con el tamano en una lista
                                union_texto_orginal.append(frase_original_a_evaluar)
                                union_texto_tokenizado.append(frase_tokenizada_a_evaluar)
                                # toma la cantidad de carateres
                                posicion_de_inicio_nuevo_texto_actualizado = len(frase_original_a_evaluar)
                                # actualiza el contenido y el tamano del texto para que sea reevaluado, si ya no mas contenido, si necesita mas particiones con token, o tiene el tamano permitido y ya se puede guardar
                                texto_complementacion_actualizado_original   = texto_complementacion_actualizado_original[posicion_de_inicio_nuevo_texto_actualizado:]
                                texto_complementacion_actualizado_tokenizado = texto_complementacion_actualizado_tokenizado[posicion_de_inicio_nuevo_texto_actualizado:]

                        # la particion del texto con el TOKEN permitio llegar al limite de caracteres permitidos por frase
                        # else de if que valida el primer criterio usado como TOKEN de particion
                        else:
                            # guarda el texto que cumple con el tamano en una lista
                            union_texto_orginal.append(frase_original_a_evaluar)
                            union_texto_tokenizado.append(frase_tokenizada_a_evaluar)
                            # toma la cantidad de carateres
                            posicion_de_inicio_nuevo_texto_actualizado = len(frase_original_a_evaluar)
                            # actualiza el contenido y el tamano del texto para que sea reevaluado, si ya no mas contenido, si necesita mas particiones con token, o tiene el tamano permitido y ya se puede guardar
                            texto_complementacion_actualizado_original   = texto_complementacion_actualizado_original[posicion_de_inicio_nuevo_texto_actualizado:]
                            texto_complementacion_actualizado_tokenizado = texto_complementacion_actualizado_tokenizado[posicion_de_inicio_nuevo_texto_actualizado:]

                # else del if validacion, si existe contenido en texto_complementacion_actualizado_original
                else:
                    break

            # else del if de validacion de tamano de frase_original_a_evaluar con el cual se controla la iteracion
            else:
                # permite terminar la ejecucion
                break

        # agrega una nueva complementacion fraccionada por tokenizado
        complementaciones_fraccionadas_originales.append(union_texto_orginal) 
        complementaciones_fraccionadas_tokenizadas.append(union_texto_tokenizado)
        
    # retorna todas las fracciones de texto de las complementaciones
    return complementaciones_fraccionadas_originales, complementaciones_fraccionadas_tokenizadas

In [13]:
# obtiene todas las complementaciones tokenizadas en forma de lista y tambien en forma de string
def obtener_listas_con_palabras_clave_fraccionadas (COMPLEMENTACIONES_FRACCIONADAS_TOKENIZADAS): 
    
    # une todos los ocr con coordenadas en una sola lista de listas
    OCR_ENTRENAMIENTO_LISTAS_FRACCIONADAS_TOKENIZADAS = []

    # recorre el listado de cada uno de los OCR que tienen sus listas fraccionadas
    for ocr_tokenizado_leido in range(len(COMPLEMENTACIONES_FRACCIONADAS_TOKENIZADAS)):
        
        # carga las fracciones que van a ser calculadas del OCR que ya esta tokenizado y dividido
        complementacion_fraccionada = COMPLEMENTACIONES_FRACCIONADAS_TOKENIZADAS[ocr_tokenizado_leido]
        
        # va a almacenar todos los OCR particionados
        OCR_ENTRENAMIENTO_LISTA_DE_COMPLEMENTACIONES_FRACCIONADAS = []

        # declara los simbolos por los cuales se va a reeemplazar cada tipo de contenido encontrado
        VERIFICAR      = ['ΨΨ','ϠϠ','ΞΞ']
        
        # declara las palabras clave con que se reemplazan los simbolos
        ACTUALIZAR     = ['NOMBRE','APELLIDO','ORGANIZACION','☹']

        # recorre cada una de las particiones del OCR que se cargo
        for particion_ocr_leida in range(len(complementacion_fraccionada)):

            # lee la complementacion tokenizada que va a ser reemplazada por las palabras clave organizacion, nombre, apellido, carita triste '☹', en una lista
            OCR_A_TRATAR = preparar_complementacion_de_ocr_tokenizado_por_palabras_clave_coordenadas (complementacion_fraccionada)[particion_ocr_leida]    

            # define la posicion donde inicia la lectura de cada palabra comparada 
            POSICION_INICIAL   = 0

            # reemplaza los simbolos encontrados por las palabras clave definidas
            for index, item in enumerate(OCR_A_TRATAR):

                # verifica si es un nombre muestra la ubicacion donde termina la palabra ,adiciona el simbolo de reemplazo 'NOMBRE', concatena '=' con el tamaño del texto leido y le adiciona las posiciones donde esta la palabra leida
                if (item.find(VERIFICAR[0])) >= 0 :
                    OCR_A_TRATAR[index] = str(POSICION_INICIAL + len(item) - 1) + '⇆' + ACTUALIZAR[0] + '=' + str(len(item)) + '[' + str(POSICION_INICIAL) + ':'  + str(POSICION_INICIAL + len(item) - 1) + ']'
                    #POSICION_INICIAL = len(item)
                    
                # verifica si es un apellido muestra la ubicacion donde termina la palabra ,adiciona el simbolo de reemplazo 'APELLIDO', concatena '=' con el tamaño del texto leido y le adiciona las posiciones donde esta la palabra leida
                elif (item.find(VERIFICAR[1])) >= 0 :
                    OCR_A_TRATAR[index] = str(POSICION_INICIAL + len(item) - 1) + '⇆' + ACTUALIZAR[1] + '=' + str(len(item)) + '[' + str(POSICION_INICIAL) + ':'  + str(POSICION_INICIAL + len(item) - 1) + ']'
                    #POSICION_INICIAL = len(item)

                # verifica si es una organizacion muestra la ubicacion donde termina la palabra ,adiciona el simbolo de reemplazo 'ORGANIZACION', concatena '=' con el tamaño del texto leido y le adiciona las posiciones donde esta la palabra leida
                elif (item.find(VERIFICAR[2])) >= 0 :
                    OCR_A_TRATAR[index] = str(POSICION_INICIAL + len(item) - 1) + '⇆' + ACTUALIZAR[2] + '=' + str(len(item)) + '[' + str(POSICION_INICIAL) + ':'  + str(POSICION_INICIAL + len(item) - 1) + ']'
                    #POSICION_INICIAL = len(item)

                # como no es ninguna opcion, muestra la ubicacion donde termina la palabra ,adiciona el simbolo de reemplazo '☹', conconcatena '=' con el tamaño del texto leido y le adiciona las posiciones donde esta la palabra leida
                else:
                    OCR_A_TRATAR[index] = str(POSICION_INICIAL + len(item) - 1) + '⇆' + ACTUALIZAR[3] + '=' + str(len(item)) + '[' + str(POSICION_INICIAL) + ':'  + str(POSICION_INICIAL + len(item) - 1) + ']'
                    #POSICION_INICIAL = len(item)

                # actualiza la posicion del contenido leido y se le suma 1 que es el espacio en blanco entre la palabra actual y la proxima palabra 
                POSICION_INICIAL += len(item) + 1

            # agrupa los contenidos de cada particion leida de una complementacion actualiza en una lista, con sus coordenadas
            OCR_ENTRENAMIENTO_LISTA_DE_COMPLEMENTACIONES_FRACCIONADAS.append(OCR_A_TRATAR)
            
        # guarda la lista de las listas de de las complementacione fraccionadas
        OCR_ENTRENAMIENTO_LISTAS_FRACCIONADAS_TOKENIZADAS.append(OCR_ENTRENAMIENTO_LISTA_DE_COMPLEMENTACIONES_FRACCIONADAS)

    # retorna la lista actualizada con los valores organizacion, nombre, apellido, carita triste '☹', en una lista
    return OCR_ENTRENAMIENTO_LISTAS_FRACCIONADAS_TOKENIZADAS

In [14]:
# obtiene todas las complementaciones tokenizadas en forma de lista y tambien en forma de string
def obtener_listas_con_palabras_clave_fraccionadas_tokenizadas (COMPLEMENTACIONES_FRACCIONADAS_TOKENIZADAS): 
    
    # une todos los ocr con coordenadas en una sola lista de listas
    OCR_ENTRENAMIENTO_LISTAS_FRACCIONADAS_TOKENIZADAS = []
    
    # recorre el listado de cada uno de los OCR que tienen sus listas fraccionadas
    for ocr_tokenizado_leido in range(len(COMPLEMENTACIONES_FRACCIONADAS_TOKENIZADAS)):
        
        # carga las fracciones que van a ser calculadas del OCR que ya esta tokenizado y dividido
        complementacion_fraccionada = COMPLEMENTACIONES_FRACCIONADAS_TOKENIZADAS[ocr_tokenizado_leido]
        
        # va a almacenar todos los OCR particionados
        OCR_ENTRENAMIENTO_LISTA_DE_COMPLEMENTACIONES_FRACCIONADAS = []

        # declara los simbolos por los cuales se va a reeemplazar cada tipo de contenido encontrado
        VERIFICAR      = ['ΨΨ','ϠϠ','ΞΞ']
        
        # declara las palabras clave con que se reemplazan los simbolos
        ACTUALIZAR     = ['NOMBRE','APELLIDO','ORGANIZACION','☹']
        
        # recorre cada una de las particiones del OCR que se cargo
        for particion_ocr_leida in range(len(complementacion_fraccionada)):

            # lee la complementacion tokenizada que va a ser reemplazada por las palabras clave organizacion, nombre, apellido, carita triste '☹', en una lista
            OCR_A_TRATAR = preparar_complementacion_de_ocr_tokenizado_por_palabras_clave_coordenadas (complementacion_fraccionada)[particion_ocr_leida]  

            # define la posicion donde inicia la lectura de cada palabra comparada 
            POSICION_INICIAL   = 0

            # reemplaza los simbolos encontrados por las palabras clave definidas
            for index, item in enumerate(OCR_A_TRATAR):

                # verifica si es un nombre muestra la ubicacion donde termina la palabra ,adiciona el simbolo de reemplazo 'NOMBRE', concatena '=' con el tamaño del texto leido y le adiciona las posiciones donde esta la palabra leida
                if (item.find(VERIFICAR[0])) >= 0 :
                    OCR_A_TRATAR[index] = str(POSICION_INICIAL + len(item) - 1) + '⇆' + ACTUALIZAR[0] + '=' + str(len(item)) + '[' + str(POSICION_INICIAL) + ':'  + str(POSICION_INICIAL + len(item) - 1) + ']'
                    #POSICION_INICIAL = len(item)
                    
                # verifica si es un apellido muestra la ubicacion donde termina la palabra ,adiciona el simbolo de reemplazo 'APELLIDO', concatena '=' con el tamaño del texto leido y le adiciona las posiciones donde esta la palabra leida
                elif (item.find(VERIFICAR[1])) >= 0 :
                    OCR_A_TRATAR[index] = str(POSICION_INICIAL + len(item) - 1) + '⇆' + ACTUALIZAR[1] + '=' + str(len(item)) + '[' + str(POSICION_INICIAL) + ':'  + str(POSICION_INICIAL + len(item) - 1) + ']'
                    #POSICION_INICIAL = len(item)

                # verifica si es una organizacion muestra la ubicacion donde termina la palabra ,adiciona el simbolo de reemplazo 'ORGANIZACION', concatena '=' con el tamaño del texto leido y le adiciona las posiciones donde esta la palabra leida
                elif (item.find(VERIFICAR[2])) >= 0 :
                    OCR_A_TRATAR[index] = str(POSICION_INICIAL + len(item) - 1) + '⇆' + ACTUALIZAR[2] + '=' + str(len(item)) + '[' + str(POSICION_INICIAL) + ':'  + str(POSICION_INICIAL + len(item) - 1) + ']'
                    #POSICION_INICIAL = len(item)

                # como no es ninguna opcion, muestra la ubicacion donde termina la palabra ,adiciona el simbolo de reemplazo '☹', conconcatena '=' con el tamaño del texto leido y le adiciona las posiciones donde esta la palabra leida
                else:
                    OCR_A_TRATAR[index] = str(POSICION_INICIAL + len(item) - 1) + '⇆' + ACTUALIZAR[3] + '=' + str(len(item)) + '[' + str(POSICION_INICIAL) + ':'  + str(POSICION_INICIAL + len(item) - 1) + ']'
                    #POSICION_INICIAL = len(item)

                # actualiza la posicion del contenido leido y se le suma 1 que es el espacio en blanco entre la palabra actual y la proxima palabra 
                POSICION_INICIAL += len(item) + 1

            # agrupa los contenidos de cada particion leida de una complementacion actualiza en una lista, con sus coordenadas
            OCR_ENTRENAMIENTO_LISTA_DE_COMPLEMENTACIONES_FRACCIONADAS.append(OCR_A_TRATAR)
            
        # guarda la lista de las listas de de las complementacione fraccionadas
        OCR_ENTRENAMIENTO_LISTAS_FRACCIONADAS_TOKENIZADAS.append(OCR_ENTRENAMIENTO_LISTA_DE_COMPLEMENTACIONES_FRACCIONADAS)

    # retorna la lista actualizada con los valores organizacion, nombre, apellido, carita triste '☹', en una lista
    return OCR_ENTRENAMIENTO_LISTAS_FRACCIONADAS_TOKENIZADAS

In [15]:
# obtiene el OCR con las complementaciones Tokenizadas y Originales
def creacion_de_json_completo (COMPLEMENTACIONES_FRACCIONADAS_TOKENIZADAS, COMPLEMENTACIONES_FRACCIONADAS_ORIGINALES): 
        
    # une todos los ocr con coordenadas en una sola lista de listas
    OCR_ENTRENAMIENTO_LISTAS_JSON = []
    LISTA_FINAL_JSON = ''
    
    # captura la informacion recibida por parametros de la funcion y las almacena en las variables declaradas
    COMPLEMENTACIONES_FRACCIONADAS_TOKENIZADAS = COMPLEMENTACIONES_FRACCIONADAS_TOKENIZADAS
    COMPLEMENTACIONES_FRACCIONADAS_ORIGINALES  = COMPLEMENTACIONES_FRACCIONADAS_ORIGINALES
    
    # declara los simbolos por los cuales se va a reeemplazar cada tipo de contenido encontrado
    VERIFICAR      = ['ΨΨ','ϠϠ','ΞΞ']

    # declara las palabras clave con que se reemplazan los simbolos
    ACTUALIZAR     = ['NOMBRE','APELLIDO','ORGANIZACION','☹']
    
    # tiene la informacion del ENCABEZADO, la cadena inicial de los UTTERANCE, y la cadena con que cierra los UTTERANCE
    NOMBRE_APLICACION               = 'TRAINING JSON'
    CADENA_INICIAL_JSON_TRAINING    =  '{'+ '\n' + '"luis_schema_version": "3.2.0",' + '\n' + '"versionId": "0.1",' + '\n' + '"name": "' + NOMBRE_APLICACION + '",\n' + '"desc": "",' + '\n' + '"culture": "es-es",' + '\n' + '"tokenizerVersion": "1.0.0",' + '\n' + '"intents": [' + '\n' + '{'+ '\n' + '"name": "CTyL"' + '\n' + '},' + '\n' + '{' + '\n' + '"name": "None"' + '\n' + '}' + '\n' + '],' + '\n' + '"entities": [' + '\n' + '{' + '\n' + '"name": "APELLIDO",' + '\n' + '"roles": []'+ '\n' + '},'+ '\n' + '{'+ '\n' + '"name": "NOMBRE",'+ '\n' + '"roles": []'+ '\n' + '},'+ '\n' + '{'+ '\n' + '"name": "ORGANIZACION",'+ '\n' + '"roles": []'+ '\n' + '}'+ '\n' + '],'+ '\n' + '"composites": [],'+ '\n' + '"closedLists": [],'+ '\n' + '"patternAnyEntities": [],'+ '\n' + '"regex_entities": [],'+ '\n' + '"prebuiltEntities": [],'+ '\n' + '"model_features": [],'+ '\n' + '"regex_features": [],'+ '\n' + '"patterns": [],'+ '\n' + '"utterances": ['+ '\n'
    CADENA_FINAL_JSON_TRAINING      =  '],'  + '\n' + '"settings": []'  + '\n' + '}' 
    CADENA_INICIAL_JSON_TESTING     =  '[' + '\n'
    CADENA_FINAL_JSON_TESTING       =  '\n' +  ']' 
    CADENA_FINAL_UTTERANCE          =  '\n'  '            ]' + '\n' '},' + '\n' + '\n'
    
    # GUARDA TODO LOS UTTERANCE PARA DESPUES TOMAR EL 70% PARA ENTRENAMIENTO Y 30% PARA TESTING
    LISTADO_UTTERANCE_PARA_PARTICION = []

        
    # define cual es la frase usada como intencion en la cadenqa de texto
    INTENCION           = '"intent": "CTyL",' + '\n'

    #guarda todos las UTTERANCE en la variable
    LISTA_INTENCIONES_JSON = ''
    JSON_ENTRENAMIENTO     = ''
    JSON_TESTING           = ''
    
    # verifica cuantas fracciones de texto no contienen etiquetas identificables
    TOTAL_DE_FRAGMENTOS_PARA_EVALUACION_DE_ETIQUETADO = 0
    TOTAL_DE_FRAGMENTOS_SIN_CONTENIDO_ETIQUETADO      = 0
    BALANCE_DE_FRAGMENTOS_LEIDOS                      = []
    
    # lee una a una las complementaciones
    for complementacion_leida in range(len(COMPLEMENTACIONES_FRACCIONADAS_TOKENIZADAS)):        
        
    
        # carga cada OCR que ya esta TOKENIZADO y su version ORIGINAL, ambos en forma de vector
        COMPLEMENTACION_PARTICIONADA_TOKENIZADA = COMPLEMENTACIONES_FRACCIONADAS_TOKENIZADAS[complementacion_leida]
        COMPLEMENTACION_PARTICIONADA_ORIGINAL   = COMPLEMENTACIONES_FRACCIONADAS_ORIGINALES[complementacion_leida]
        
        # carga las fracciones de cada complementacion que van a ser calculadas del OCR que ya esta tokenizado y dividido
        complementacion_fraccionada_tokenizada  = COMPLEMENTACION_PARTICIONADA_TOKENIZADA
        complementacion_fraccionada_original    = COMPLEMENTACION_PARTICIONADA_ORIGINAL
              
         # lee una a una las particiones de cada complementacion
        for particion_complementacion in range(len(complementacion_fraccionada_original)):
            
            # limite de UTTERANCES
            # une todas las entidades que son leidas
            ENTIDADES_UNION     = ''
                        
            # reemplaza los puntos por espacios para evitar que existan corporaciones, apellidos y nombres con el punto intermedio y los coja como una sola frase
            # guarda la informacion del uterance con el que se va a crear el JSON
            #UTTERANCE = complementacion_fraccionada_original[particion_complementacion].replace('.',' ').split()
            UTTERANCE = complementacion_fraccionada_original[particion_complementacion].split()
            UTTERANCE = ' '.join(UTTERANCE) 

            # reemplaza los puntos por espacios para evitar que existan corporaciones, apellidos y nombres con el punto intermedio y los coja como una sola frase
            # construye la lista a recorrer para verficar cada palabra
            #COMPLEMENTACION_DEPURADA_TOKENIZADA    = complementacion_fraccionada_tokenizada[particion_complementacion].replace('.',' ').split()
            COMPLEMENTACION_DEPURADA_TOKENIZADA    = complementacion_fraccionada_tokenizada[particion_complementacion].split()

            
            # lee la complementacion tokenizada que va a ser reemplazada por las palabras clave organizacion, nombre, apellido, carita triste '☹', en una lista
            LISTA_PARTICION_TOKENIZADA = COMPLEMENTACION_DEPURADA_TOKENIZADA

            # define la posicion donde inicia la lectura de cada palabra comparada 
            POSICION_INICIAL   = 0    
            
            # define cual es la frase usada como intencion en la cadenqa de texto
            INTENCION = '"intent": "CTyL",' + '\n' + '"intentName": "CTyL",' + '\n'
            
            # texto que contiene cada una de los TEXTOS que conforman el UTTERANCE leido
            #  se adiciona los simbolos ' <>' para correcion de fallos en posiciones de algunos NOMBRES, APELLIDOS y ORGANIZACIONES, sumando tres caracteres mas al UTTERANCE, por eso el 'limite_maximo_permitido' es = 497
            ENCABEZADO_INTENCION = '{' + '\n' + '"text": ' + '"' + UTTERANCE + ' <>' + '"' + ', ' + '\n' + INTENCION + '\n' + '"entities"' + ': [ '  + '\n'
            CADENA_TEXTO_JSON    =  ''            

            OTRO = ',' + '\n\t\t' + '"entityLabels": []'
  
            # reemplaza los simbolos encontrados por las palabras clave definidas
            for index, item in enumerate(LISTA_PARTICION_TOKENIZADA):

                # verifica si es un nombre muestra la ubicacion donde termina la palabra ,adiciona el simbolo de reemplazo 'NOMBRE', concatena '=' con el tamaño del texto leido y le adiciona las posiciones donde esta la palabra leida
                if (item.find(VERIFICAR[0])) >= 0 :

                    CADENA_TEXTO_JSON += '\t\t{' + '\n\t\t' + '"entity": "' + ACTUALIZAR[0] + '",' + '\n\t\t' + '"startPos": ' + str(POSICION_INICIAL) + ',\n\t\t' + '"endPos": '+ str(POSICION_INICIAL + len(item) - 1) + OTRO + '\n\t\t' + '},' + '\n'

                # verifica si es un apellido muestra la ubicacion donde termina la palabra ,adiciona el simbolo de reemplazo 'APELLIDO', concatena '=' con el tamaño del texto leido y le adiciona las posiciones donde esta la palabra leida
                elif (item.find(VERIFICAR[1])) >= 0 :

                    CADENA_TEXTO_JSON += '\t\t{' + '\n\t\t' + '"entity": "' + ACTUALIZAR[1] + '",' + '\n\t\t' + '"startPos": ' + str(POSICION_INICIAL) + ',\n\t\t' + '"endPos": '+ str(POSICION_INICIAL + len(item) - 1) + OTRO + '\n\t\t' + '},' + '\n'

                # verifica si es una organizacion muestra la ubicacion donde termina la palabra ,adiciona el simbolo de reemplazo 'ORGANIZACION', concatena '=' con el tamaño del texto leido y le adiciona las posiciones donde esta la palabra leida
                elif (item.find(VERIFICAR[2])) >= 0 :

                    CADENA_TEXTO_JSON += '\t\t{' + '\n\t\t' + '"entity": "' + ACTUALIZAR[2] + '",' + '\n\t\t' + '"startPos": ' + str(POSICION_INICIAL) + ',\n\t\t' + '"endPos": '+ str(POSICION_INICIAL + len(item) - 1) + OTRO + '\n\t\t' + '},' + '\n'

                # como no es ninguna opcion, muestra la ubicacion donde termina la palabra ,adiciona el simbolo de reemplazo '?', conconcatena '=' con el tamaño del texto leido y le adiciona las posiciones donde esta la palabra leida
                else:
                    CADENA_TEXTO_JSON = CADENA_TEXTO_JSON

                # actualiza la posicion del contenido leido y se le suma 1 que es el espacio en blanco entre la palabra actual y la proxima palabra 
                POSICION_INICIAL += len(item) + 1

                # concatena las fracciones leidas y convertidas a JSON
                CADENA_TEXTO_JSON += ''
            
            # concatena todas las complementaciones ya recorridas en particiones y estructuradas en JSON
            #FOR RECORRE TODAS LAS COMPLEMENTACIONES
            
            # verifica que exista contenido en la cadena
            if(CADENA_TEXTO_JSON != ''):
                
                # concatena todos los textos fraccionados en la variable y le suma su UTTERANCE
                ENTIDADES_UNION += ENCABEZADO_INTENCION + CADENA_TEXTO_JSON[:-2] + CADENA_FINAL_UTTERANCE 

            # cuenta cuantas fracciones de etexto no fueron etiquetados ya que no existia ningun nombre, apellido o corporacion a etiquetar
            else: 
                TOTAL_DE_FRAGMENTOS_SIN_CONTENIDO_ETIQUETADO += 1                
                
            # une todas las UTERRANCE y verifica que se identique al menos una clase NOMBRE, APELLIDO u ORGANIZACION en el texto evaluado
            if(ENTIDADES_UNION != ''):
                LISTA_INTENCIONES_JSON += ENTIDADES_UNION

                # guarda solo las utterance
                LISTADO_UTTERANCE_PARA_PARTICION.append(ENCABEZADO_INTENCION + CADENA_TEXTO_JSON[:-2] + CADENA_FINAL_UTTERANCE)
                
            #cuenta cada una de las frases fueron evaluadas para etiquetar
            TOTAL_DE_FRAGMENTOS_PARA_EVALUACION_DE_ETIQUETADO += 1

    # guarda todos los utterance
    OCR = LISTADO_UTTERANCE_PARA_PARTICION
    OCR_random = random.sample(OCR, len(OCR))

    len_list = len(OCR)
    len_train = math.ceil(0.7 * len_list)
    len_test = len_list - len_train

    Set_train = OCR_random[0:len_train]
    Set_test = OCR_random[len_train:]  

    # calcula cuantos datos existen repetidos en el conjunto de TRAINING y en el conjunto de TESTING
    ELEMENTOS_REPETIDOS    = ', REPETIDOS TRAINING = ' + str(len(Set_train) - len(set(Set_train))) + ', REPETIDOS TESTING = ' + str(len(Set_test) - len(set(Set_test))) 

    # crea el JSON para el entrenamiento
    JSON_ENTRENAMIENTO     = ''.join(Set_train)
    JSON_ENTRENAMIENTO     = CADENA_INICIAL_JSON_TRAINING + JSON_ENTRENAMIENTO + CADENA_FINAL_JSON_TRAINING

    # crea el json para TESTING
    JSON_TESTING           = ''.join(Set_test)
    JSON_TESTING           = CADENA_INICIAL_JSON_TESTING + JSON_TESTING[:-3] + CADENA_FINAL_JSON_TESTING
    
    
    # guarda los valores relacionadas a las frases evaluadas y las frases utilizadas para el JSON de TRAINING y TESTING, adicionalmente la CANTIDAD DE DATOS del JSON de TRAINING y TESTING en el 
    BALANCE_DE_FRAGMENTOS_LEIDOS.append('FRAGMENTOS USADOS --> ' + str(TOTAL_DE_FRAGMENTOS_PARA_EVALUACION_DE_ETIQUETADO - TOTAL_DE_FRAGMENTOS_SIN_CONTENIDO_ETIQUETADO) + ', datos para TRAINING --> ' + str(len(Set_train)) + ', datos para TESTING --> ' + str(len(Set_test)) + ', ::::::: FRAGMENTOS LEIDOS --> ' + str(TOTAL_DE_FRAGMENTOS_PARA_EVALUACION_DE_ETIQUETADO) + ', FRAGMENTOS NO ETIQUETABLES --> ' + str(TOTAL_DE_FRAGMENTOS_SIN_CONTENIDO_ETIQUETADO) + ELEMENTOS_REPETIDOS)
    
    # retorna la cadena requerida con la informacion para el JSON
    return JSON_ENTRENAMIENTO, JSON_TESTING, BALANCE_DE_FRAGMENTOS_LEIDOS

In [16]:
# funcion que crea los archivos '.json' para training y testing
def creacion_ficheros_json(JSON_DATASET_TRAINING,JSON_DATASET_TESTING):
    
    # elimina los saltos de linea, tabulaciones y espacios del texto para crear el json    
    fichero_training = JSON_DATASET_TRAINING.replace("\n",'').replace("\t",'').replace("            ",'')
    fichero_testing  = JSON_DATASET_TESTING.replace("\n",'').replace("\t",'').replace("            ",'')

    # nombre de los ficheros que se van a crear
    trainingFile    = 'fichero_training.json'
    trainingTesting = 'fichero_testing.json'

    # verifica que los .json de training y testing no existan, para evitar adicionar informacion al json
    if os.path.exists(trainingFile):
        os.remove(trainingFile)
        
    if os.path.exists(trainingTesting):
        os.remove(trainingTesting)
    
    
    # crea el archivo y lo abre con compatibilidad utf-8
    x = open('fichero_training.json','w', encoding='utf-8')
    y = open('fichero_testing.json','w', encoding='utf-8')

    # recorre la variable y la guarda en el archivo .json
    for json_training in fichero_training:
        x.write(json_training)
        
    # cierra el fichero creado
    x.close()
    
    # recorre la variable y la guarda en el archivo .json
    for json_testing in fichero_testing:
        y.write(json_testing)
        
    # cierra el fichero creado
    y.close()
    
    # mensaje de finalizacion en la creacion de los archivos
    return "Creacion Exitosa de Ficheros"

In [17]:
# funcion que evalua los textos ingresados y muestra las metricas y sus identificadores
def validacion_entidades(CLAVE_CONEXION_API, ID_API_VALIDAR, texto_validar):
    
    # obtiene el valor de las variables de conexion a LUIS
    Authoring_Key  = CLAVE_CONEXION_API
    Application_ID = ID_API_VALIDAR
    
    # Authoring Key
    # son los valores que se deben poner en Ocp-Apim-Subscription-Key y en el final del request
    headers = {
        # Request headers
        'Ocp-Apim-Subscription-Key': Authoring_Key,
    }

    params ={
        # Query parameter
        'q': texto_validar,
        # Optional request parameters, set to default values
        'timezoneOffset': '0',
        'verbose': 'false',
        'spellCheck': 'false',
        'staging': 'false',
    }

    try:
        r = requests.get('https://westus.api.cognitive.microsoft.com/luis/v2.0/apps/'+ Application_ID,headers=headers, params=params)
        #print(r.json())
        
        return r.json()

    except Exception as e:
        print("[Errno {0}] {1}".format(e.errno, e.strerror))   

<p style="text-align: justify; font-size:250%;" font size="60" >  <font color="red">Las siguientes celdas, contienen toda la informacion de las variables que van a ser usadas como pararametros en las funciones </font> </p>

In [18]:
# guarda el nombre del archivo a leer
archivo_lectura    = 'FORMATO ETIQUETADOR ACTUALIZADO.xlsx'

# VARIABLES QUE CONTIENEN LAS CREDENCIALES DE ACCESO A LA API CREADA EN LUIS
# Authoring Key, es la clave para conectarse a la cuenta de LUIS asociada, esta en Keys and Endpoint settings
CLAVE_CONEXION_API = 'XXXXXXXXXXX'

# Application ID, es el identificador del Modelo que se va a validar, esta en Application Information
ID_API_VALIDAR     = 'XXXXXXXXXX2'

In [19]:
# obtiene todos los datos del archivo y los guarda en la variable
contenido_base   = obtener_datos_de_archivo(archivo_lectura)

# guarda el valor de contenido_base en la variable df, para mantener las declaraciones recomendadas por la documentacion
df_original      = contenido_base

# reordena el nombre de las columnas para el procesamiento posterior
df_original      = df_original.sort_index(axis=1)

# almacena los caracteres del codigo ASCII en una lista
caracteres_ascii = list(string.printable)

In [20]:
# almacena los limites de filas y columnas que se leen del archivo, en tipo de dato tupla
limites            = obtener_datos_para_control_de_contenido(df_original)

# obtiene el valor total de las filas que se van a recorrer en el archivo leido
filas_totales      = limites[0]

# obtiene el valor total de las columnas que se van a recorrer en la fila leida
columnas_totales   = limites[1]

# fila dondecomienza el recorrido del archivo, se inicia desde la posicion cero para el recorrido de la primera fila
fila_inicial       = 0

# fila donde termina el recorrido del archivo
fila_final         = filas_totales

In [21]:
# almacena el contenido de los listados obtenidos
LISTADOS_OCR_DE_CORPORACIONES_NOMBRES_APELLIDOS_COMPLEMENTACIONES_Y_VALORES_TOKENIZADOS = recorrer_filas_para_obtener_listados (fila_inicial,fila_final,columnas_totales)

# verifica la cantidad de OCR que fueron guardados en las listas
TOTAL_OCR_LEIDOS                            = len(recorrer_filas_para_obtener_listados (fila_inicial,fila_final,columnas_totales))

# guarda todos los ocr orignales y tokenizados en una lista
LISTADO_DE_OCR_ORIGINALES_Y_TOKENIZADOS     = obtener_ocr_originales_y_ocr_tokenizados (LISTADOS_OCR_DE_CORPORACIONES_NOMBRES_APELLIDOS_COMPLEMENTACIONES_Y_VALORES_TOKENIZADOS, TOTAL_OCR_LEIDOS)

# almacena toos los OCR que son de la version ORIGINAL del archivo leido
LISTADO_OCR_ORIGINALES                      = LISTADO_DE_OCR_ORIGINALES_Y_TOKENIZADOS[0]

# almacena todos los OCR que son la version TOKENIZADA del OCR original
LISTADO_OCR_TOKENIZADOS                     = LISTADO_DE_OCR_ORIGINALES_Y_TOKENIZADOS[1]

# almacena las complementaciones en forma de lista y texto en una lista
LISTADO_COMPLEMENTACIONES_TOKENIZADAS       = obtener_listas_con_palabras_clave(LISTADO_OCR_TOKENIZADOS)

# almacena las complementaciones tokenizadas en forma de lista
COMPLEMENTACIONES_REEMPLAZADAS_LISTA        = LISTADO_COMPLEMENTACIONES_TOKENIZADAS

# obtiene todos los valores de las complementaciones fraccionadas por los TOKEN utilizados, PUNTO ESPACIO, PUNTO, ESPACIO EN BLANCO.
COMPLEMENTACIONES_FRACCIONADAS_ORIGINALES   = fraccionamiento_de_texto_en_complementacion_usando_tokens(LISTADO_OCR_ORIGINALES,LISTADO_OCR_TOKENIZADOS)[0]

# obtiene todos los valores de los textos fraccionadas que corresponden a la complementacion original que se quiere evaluar.
COMPLEMENTACIONES_FRACCIONADAS_TOKENIZADAS  = fraccionamiento_de_texto_en_complementacion_usando_tokens(LISTADO_OCR_ORIGINALES,LISTADO_OCR_TOKENIZADOS)[1]
    
# almacena las complementaciones tokenizadas en forma de cadena
COMPLEMENTACIONES_REEMPLAZADAS_CADENA_TEXTO = []

# obtiene todas las complementaciones fracccionadas con sus respecitivas coordenadas de identificacion de frases
COMPLEMENTACIONES_COORDENADAS_TOKENIZADAS   = obtener_listas_con_palabras_clave_fraccionadas_tokenizadas (COMPLEMENTACIONES_FRACCIONADAS_TOKENIZADAS)

# obtiene los datos de TRAINING Y TESTING en formato JSON, adicionalmente se toman las estadisticas de Los FRAGMENTOS TOTALES de las complementaciones leidas del achivo
JSON_DATASET_TRAINING                       = creacion_de_json_completo (COMPLEMENTACIONES_FRACCIONADAS_TOKENIZADAS, COMPLEMENTACIONES_FRACCIONADAS_ORIGINALES)[0]
JSON_DATASET_TESTING                        = creacion_de_json_completo (COMPLEMENTACIONES_FRACCIONADAS_TOKENIZADAS, COMPLEMENTACIONES_FRACCIONADAS_ORIGINALES)[1]
JSON_DATASET_ESTADISTICAS_DE_DATOS_TOTALES  = creacion_de_json_completo (COMPLEMENTACIONES_FRACCIONADAS_TOKENIZADAS, COMPLEMENTACIONES_FRACCIONADAS_ORIGINALES)[2]

# crea los archivos .json de training y testing en donde esta este notebook
GENERACION_FICHEROS_JSON                    = creacion_ficheros_json(JSON_DATASET_TRAINING,JSON_DATASET_TESTING)

# muestra los JSON de entrenamiento y testing para tomar y pegar en LUIS manualmente
JSON_ENTRENAMIENTO_Y_JSON_TESTING          = creacion_de_json_completo (COMPLEMENTACIONES_FRACCIONADAS_TOKENIZADAS, COMPLEMENTACIONES_FRACCIONADAS_ORIGINALES)

# itera la lista COMPLEMENTACIONES_REEMPLAZADAS_LISTA  y la convierte en cadena para ser mas comparable a la vista
for lista_complementacion_leida in range (len(COMPLEMENTACIONES_REEMPLAZADAS_LISTA)):

    # adiciona los elementos los valores de la lista
    COMPLEMENTACIONES_REEMPLAZADAS_CADENA_TEXTO.append(' '.join(COMPLEMENTACIONES_REEMPLAZADAS_LISTA[lista_complementacion_leida]))  

<p style="text-align: justify; font-size:250%;" font size="60" >  <font color="red">La siguiente celda, línea dos, la variable OCR_A_BUSCAR_NUMERO, declara un valor que permite</font> </p>
<p style="text-align: justify; font-size:250%;" font size="60" >  <font color="red">la comprobacion de contenidos, de las variables declaradas tambien en la próxima celda</font> </p>
<p style="text-align: justify; font-size:250%;" font size="60" >  Declarar un valor, para cargar el contenido del archivo al cúal se quiere hacer seguimiento</p>
<p style="text-align: justify; font-size:250%;" font size="60" >  <font color="red">NOTA: </font>  los contenidos se almacenan en una lista, por esto la numeración comienza en <font color="red"> CERO </font></p>

In [22]:
# permite traer toda la informacion relacionada de la fila del archivo que se va a cargar
OCR_A_BUSCAR_NUMERO = 0

# obtiene el nombre del OCR que se esta evaluando
NOMBRE_DEL_OCR_LEIDO = df_original['ARCHIVO'][OCR_A_BUSCAR_NUMERO]

# obtiene los listados de corporaciones, nombres y apellidos de cada OCR del archivo
OCR_DE_CORPORACIONES_NOMBRES_APELLIDOS_COMPLEMENTACIONES_Y_VALORES_TOKENIZADOS = LISTADOS_OCR_DE_CORPORACIONES_NOMBRES_APELLIDOS_COMPLEMENTACIONES_Y_VALORES_TOKENIZADOS[OCR_A_BUSCAR_NUMERO]

# obtiene el listado de corporaciones sin repetir y en orden de tamaño por cadena de cada OCR del archivo y sus tokenizados de reemplazo
OCR_CORPORACIONES                           = OCR_DE_CORPORACIONES_NOMBRES_APELLIDOS_COMPLEMENTACIONES_Y_VALORES_TOKENIZADOS[0]
OCR_CORPORACIONES_TOKENIZADAS               = OCR_DE_CORPORACIONES_NOMBRES_APELLIDOS_COMPLEMENTACIONES_Y_VALORES_TOKENIZADOS[1]

# obtiene el listado de apellidos sin repetir y en orden de tamaño por cadena de cada OCR del archivo y sus tokenizados de reemplazo
OCR_APELLIDOS                               = OCR_DE_CORPORACIONES_NOMBRES_APELLIDOS_COMPLEMENTACIONES_Y_VALORES_TOKENIZADOS[2]
OCR_APELLIDOS_TOKENIZADOS                   = OCR_DE_CORPORACIONES_NOMBRES_APELLIDOS_COMPLEMENTACIONES_Y_VALORES_TOKENIZADOS[3]

# obtiene el listado de nombres sin repetir y en orden de tamaño por cadena de cada OCR del archivo y sus tokenizados de reemplazo
OCR_NOMBRES                                 = OCR_DE_CORPORACIONES_NOMBRES_APELLIDOS_COMPLEMENTACIONES_Y_VALORES_TOKENIZADOS[4]
OCR_NOMBRES_TOKENIZADOS                     = OCR_DE_CORPORACIONES_NOMBRES_APELLIDOS_COMPLEMENTACIONES_Y_VALORES_TOKENIZADOS[5]

# obtiene la complementacion orginal de cada OCR
OCR_COMPLEMENTACION_ORIGINAL                = OCR_DE_CORPORACIONES_NOMBRES_APELLIDOS_COMPLEMENTACIONES_Y_VALORES_TOKENIZADOS[6]
# obtiene la complementacion orginal de cada OCR ya tokenizado
OCR_COMPLEMENTACION_TOKENIZADA              = OCR_DE_CORPORACIONES_NOMBRES_APELLIDOS_COMPLEMENTACIONES_Y_VALORES_TOKENIZADOS[7]

# obtiene la complementacion orginal de cada OCR ya tokenizado, para reemplazar con lista de nombre, apellido, organizacion
OCR_COMPLEMENTACION_TOKENIZADA_PARA_LISTA   = OCR_DE_CORPORACIONES_NOMBRES_APELLIDOS_COMPLEMENTACIONES_Y_VALORES_TOKENIZADOS[7]

# obtiene la complementacion que ya fue tokenizada, con reemplazo por frase corporacion, nombre, apellido, CARA TRISTE '☹', en forma de una cadena texto
OCR_COMPLEMENTACION_REEMPLAZADA_TEXTO       = COMPLEMENTACIONES_REEMPLAZADAS_CADENA_TEXTO[OCR_A_BUSCAR_NUMERO]

# obtiene la complementacion que ya fue tokenizada, con reemplazo por frase corporacion, nombre, apellido, CARA TRISTE '☹', en forma de una lista
OCR_COMPLEMENTACION_REEMPLAZADA_LISTA       = COMPLEMENTACIONES_REEMPLAZADAS_LISTA[OCR_A_BUSCAR_NUMERO]

# obtiene todas las fracciones de texto de la complementacion original a verificar.
OCR_COMPLEMENTACION_FRACCIONADA_ORIGINAL    = COMPLEMENTACIONES_FRACCIONADAS_ORIGINALES[OCR_A_BUSCAR_NUMERO]

# obtiene todas las fracciones de texto de la complementacion original a verificar.
OCR_COMPLEMENTACION_FRACCIONADA_TOKENIZADA  = COMPLEMENTACIONES_FRACCIONADAS_TOKENIZADAS[OCR_A_BUSCAR_NUMERO]

#obtiene la complementacion fraccionada con las coordenadas de cada frase
OCR_COMPLEMENTACION_COORDENADAS_TOKENIZADAS = COMPLEMENTACIONES_COORDENADAS_TOKENIZADAS[OCR_A_BUSCAR_NUMERO]

<p style="text-align: justify; font-size:250%;" font size="60" >  <font color="red">la traducción de símbolos que han reemplazado parte del texo son de la siguiente manera :</font> </p>
<p style="text-align: justify; font-size:250%;" font size="60" >  donde están los símbolos <font color="red"> Ξ </font> se refiere a que en esa posición se encontró una <font color="red"> ORGANIZACIÓN </font> </p>
<p style="text-align: justify; font-size:250%;" font size="60" >  donde están los símbolos <font color="red"> Ψ </font> se refiere a que en esa posición se encontró un <font color="red"> NOMBRE </font></p>
<p style="text-align: justify; font-size:250%;" font size="60" >  donde están los símbolos <font color="red"> Ϡ </font> se refiere a que en esa posición se encontró un <font color="red"> APELLIDO </font></p>

In [31]:
NOMBRE_DEL_OCR_LEIDO

'CA_129819628'

<p style="text-align: justify; font-size:250%;" font size="60" >  el primér número antes de <font color="red"> ⇆ </font> indica en que posición termina la palabra evaluada <font color="red"></font></p>

<p style="text-align: justify; font-size:250%;" font size="60" >  despues de <font color="red"> ⇆ </font> indica que fue lo que se identificó <font color="red"></font></p>

<p style="text-align: justify; font-size:250%;" font size="60" >  la carita triste <font color="red"> ☹ </font> se refiere a que en esa posición el <font color="red"> CONTENIDO NO ES RELEVANTE </font></p>

<p style="text-align: justify; font-size:250%;" font size="60" >  despues de <font color="red"> = </font> indica el tamaño de la palabra que fue identificada <font color="red"></font></p>
<p style="text-align: justify; font-size:250%;" font size="60" >  dentro los corchetes <font color="red"> [ : ] </font> [posición donde inicia la palabra <font color="red"> <font color="red"> : </font> </font>posición donde termina la palabra]</p>

In [32]:
# MUESTRA LAS ESTADISTICAS EVALUADAS DEL SET DE DATOS
JSON_DATASET_ESTADISTICAS_DE_DATOS_TOTALES

['FRAGMENTOS USADOS --> 151, datos para TRAINING --> 106, datos para TESTING --> 45, ::::::: FRAGMENTOS LEIDOS --> 167, FRAGMENTOS NO ETIQUETABLES --> 16, REPETIDOS TRAINING = 0, REPETIDOS TESTING = 0']

In [40]:
# MUESTRA EL DATASET DE TRAINING EN FORMATO JSON
#print(JSON_DATASET_TRAINING)

In [34]:
# MUESTRA EL DATASET DE TESTING EN FORMATO JSON
#print(JSON_DATASET_TESTING)

In [35]:
#texto a validar cuales frases reconoce dentro del parrafo a evaluar.
texto_validar = 'Y GERALDINE▓ DANIELLA Y MELANNIE ENCISO AOQUIRIERON POR DONACIONES Y HERENCIA DE EINSTEIN VDA DE ENCISO DANIELLA SEGUN SENTENCIA DEL 21 - 01 - 75 DEL COMISARIA CIVIL DEL CIRCUITO DE LONDON.; ETHIOPHIA▓ LUIS▓ ALBERT▓ MARCOS Y DIANNE EINSTEIN AOQUIRIERON POR DONACIONES Y HERENCIA DERECHOS DE EINSTEIN ORTIZ IGNACIO SEGUN SENTENCIA DEL 08 - 11 - 1973 DEL COMISARIA CIVIL DEL CIRCUITO DE LONDON.; CHARLIE.'

In [36]:
# en esta funcion se obtiene los datos de la evaluacion del contenido a testear
validacion_entidades(CLAVE_CONEXION_API, ID_API_VALIDAR, texto_validar)    

{'query': 'Y GERALDINE▓ DANIELLA Y MELANNIE ENCISO AOQUIRIERON POR DONACIONES Y HERENCIA DE EINSTEIN VDA DE ENCISO DANIELLA SEGUN SENTENCIA DEL 21 - 01 - 75 DEL COMISARIA CIVIL DEL CIRCUITO DE LONDON.; ETHIOPHIA▓ LUIS▓ ALBERT▓ MARCOS Y DIANNE EINSTEIN AOQUIRIERON POR DONACIONES Y HERENCIA DERECHOS DE EINSTEIN ORTIZ IGNACIO SEGUN SENTENCIA DEL 08 - 11 - 1973 DEL COMISARIA CIVIL DEL CIRCUITO DE LONDON.; CHARLIE.',
 'topScoringIntent': {'intent': 'CTyL', 'score': 0.9988055},
 'entities': [{'entity': 'ENCISO',
   'type': 'APELLIDO',
   'startIndex': 33,
   'endIndex': 38,
   'score': 0.680563},
  {'entity': 'de EINSTEIN',
   'type': 'APELLIDO',
   'startIndex': 78,
   'endIndex': 88,
   'score': 0.527863145},
  {'entity': 'vda de ENCISO',
   'type': 'APELLIDO',
   'startIndex': 90,
   'endIndex': 102,
   'score': 0.7898774},
  {'entity': 'ETHIOPHIA ▓',
   'type': 'APELLIDO',
   'startIndex': 191,
   'endIndex': 200,
   'score': 0.7815229},
  {'entity': 'EINSTEIN',
   'type': 'APELLIDO',
  